In [1]:
import pandas as pd
import numpy as np
import random
import os

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.metrics import mean_absolute_percentage_error
import warnings
warnings.filterwarnings("ignore")

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

# 한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/H2GTRM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

# feature importance 
def vis_feature_importances_(est, X_train):
    ftr_importances_values = est.feature_importances_
    ftr_importances = pd.Series(ftr_importances_values,index=X_train.columns  )
    ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]
    plt.figure(figsize=(10,4))
    plt.title('Feature importances Top 20')
    sns.barplot(x=ftr_top20 , y = ftr_top20.index)
    plt.show()
    
# https://dacon.io/forum/401657    
def smape(true, pred):
    v = 2 * abs(pred - true) / (abs(pred) + abs(true))
    output = np.mean(v) * 100
    return output    

def smape_model():
    def smape(true, pred):
        v = 2 * abs(pred - true) / (abs(pred) + abs(true))
        output = np.mean(v) * 100
        return output    

## Data Load

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

# train = pd.read_csv('/content/drive/MyDrive/work/input/train.csv')
# test = pd.read_csv('/content/drive/MyDrive/work/input/test.csv')
# building_info = pd.read_csv('/content/drive/MyDrive/work/input/building_info.csv')
# submission = pd.read_csv('/content/drive/MyDrive/work/input/sample_submission.csv')
# train.shape, test.shape, building_info.shape, submission.shape

train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
building_info = pd.read_csv('input/building_info.csv')
submission = pd.read_csv('input/sample_submission.csv')
train.shape, test.shape, building_info.shape, submission.shape

((204000, 10), (16800, 7), (100, 7), (16800, 2))

## Train Data Pre-Processing

In [4]:
#시계열 특성을 학습에 반영하기 위해 일시를 월, 일, 시간으로 나눕니다
train['month'] = train['일시'].apply(lambda x : int(x[4:6]))
train['day'] = train['일시'].apply(lambda x : int(x[6:8]))
train['time'] = train['일시'].apply(lambda x : int(x[9:11]))

test['month'] = test['일시'].apply(lambda x : int(x[4:6]))
test['day'] = test['일시'].apply(lambda x : int(x[6:8]))
test['time'] = test['일시'].apply(lambda x : int(x[9:11]))

# 요일
train['weekday'] =  pd.to_datetime(train['일시'].str[:8]).dt.weekday
test['weekday'] =  pd.to_datetime(test['일시'].str[:8]).dt.weekday

In [5]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

kmeans = pd.DataFrame()

for i in range(1, 101):
#     print(i)
    temp = train[train['건물번호'] == i]
    temp['date'] = pd.to_datetime(temp['일시']).dt.strftime('%Y%m%d')
    temp = temp[['date','weekday','time', '전력소비량(kWh)']]
    temp['kmeans'] = KMeans(n_clusters=2, random_state=42).fit_predict(temp)
    temp['건물번호'] = i
    kmeans = pd.concat([kmeans, temp.reset_index()[['건물번호','weekday','time', 'kmeans']]])
kmeans = kmeans.groupby(['건물번호','weekday','time'])['kmeans'].mean().reset_index()        

kmeans['kmeans_o'] = np.where(kmeans['kmeans'] > 0.6, 1, 0)
train = train.merge(kmeans[['건물번호', 'weekday','time','kmeans_o']], on=['건물번호', 'weekday','time'])
test = test.merge(kmeans[['건물번호', 'weekday','time','kmeans_o']], on=['건물번호', 'weekday','time'])

In [6]:
# 빌딩 데이터 정리
## null 처리
building_info['태양광용량(kW)'] = building_info['태양광용량(kW)'].str.replace('-','0').astype(float)
building_info['ESS저장용량(kWh)'] = building_info['ESS저장용량(kWh)'].str.replace('-','0').astype(float)
building_info['PCS용량(kW)'] = building_info['PCS용량(kW)'].str.replace('-','0').astype(float)

In [7]:
# 정리된 빌딩과 merge
train = train.merge(building_info, on='건물번호')
test = test.merge(building_info, on='건물번호')

In [8]:
# outlier 제거 
train = train.drop(train[train['num_date_time'].isin(['56_20220608 16'
                                                  ,'34_20220808 21'
                                                  ,'10_20220607 06','10_20220607 06'
                                                  ,'31_20220808 13','31_20220809 13', '31_20220808 14','31_20220808 13'
                                                  , '8_20220806 05', '8_20220707 08', '8_20220628 09'
                                                  ,'13_20220606 16','13_20220724 06','13_20220606 17','13_20220607 07'
                                                  ,'58_20220804 16'
                                                  ,'65_20220807 18'
                                                  ,'68_20220706 20','68_20220706 19'
                                                  ,'71_20220604 10','71_20220810 15','71_20220810 16'
                                                  , '73_20220707 12', '73_20220707 14', '73_20220707 11','73_20220707 13', '73_20220707 15', '73_20220808 13','73_20220803 14', '73_20220808 12', '73_20220803 15','73_20220808 11'
                                                 ])].index)

In [9]:
train['강수량(mm)'].fillna(0, inplace=True)
train['풍속(m/s)'].fillna(train['풍속(m/s)'].mean(), inplace=True)
train['습도(%)'].fillna(train['습도(%)'].mean(), inplace=True)

test['강수량(mm)'].fillna(0, inplace=True)
test['풍속(m/s)'].fillna(test['풍속(m/s)'].mean(), inplace=True)
test['습도(%)'].fillna(test['습도(%)'].mean(), inplace=True)

In [10]:
############## 체감온도, https://www.weather.go.kr/plus/life/li_asset/HELP/basic/help_01_07.jsp
train['체감온도'] = 13.12 + 0.6215*train['기온(C)'] - 11.37*(train['풍속(m/s)']*3.6)**0.16 + 0.3965*(train['풍속(m/s)']*3.6)**0.16*train['기온(C)']
test['체감온도'] = 13.12 + 0.6215*test['기온(C)'] - 11.37*(test['풍속(m/s)']*3.6)**0.16 + 0.3965*(test['풍속(m/s)']*3.6)**0.16*test['기온(C)']

In [11]:
no_work_day = train.groupby(['건물번호', '건물유형','month', 'day']).agg({'전력소비량(kWh)':['mean','max']}).reset_index()
no_work_day.columns = ['건물번호', '건물유형','month', 'day', 'khw_mean', 'khw_max']
total_mean = no_work_day[no_work_day['건물유형'].isin(['백화점및아울렛','할인마트'])].groupby(['건물번호']).khw_mean.mean().reset_index().rename(columns={'khw_mean':'b_mean'})

no_work_day = no_work_day.merge(total_mean, on='건물번호')
no_work_day.loc[no_work_day.b_mean > no_work_day.khw_max, '백화점휴무일'] = 1

train = train.merge(no_work_day[['건물번호', '건물유형','month', 'day', '백화점휴무일']], on=['건물번호', '건물유형','month', 'day'], how='left')
train['백화점휴무일'] = train['백화점휴무일'].fillna(0).astype(int)
test['백화점휴무일'] = 0

In [12]:
train.drop( train[(train['건물번호'].isin([1,16])) & (train['month'] == 6)].index, inplace=True)

In [13]:
from feature_engine.creation import CyclicalFeatures
cf = CyclicalFeatures()
col = ['time', '기온(C)']
train = pd.concat([train, cf.fit_transform(train[col]).drop(col, axis=1)], axis=1)
test = pd.concat([test, cf.fit_transform(test[col]).drop(col, axis=1)], axis=1)
train.shape, test.shape

((202530, 27), (16800, 24))

In [14]:
from sklearn.preprocessing import SplineTransformer
spline = SplineTransformer(degree=2, n_knots=3)
col = ['풍속(m/s)','냉방면적(m2)']
train = pd.concat([train.reset_index(drop=True), pd.DataFrame(spline.fit_transform(train[col])).reset_index(drop=True)], axis=1)
test = pd.concat([test.reset_index(drop=True), pd.DataFrame(spline.fit_transform(test[col])).reset_index(drop=True)], axis=1)

train.shape, test.shape

((202530, 35), (16800, 32))

In [15]:
days = [18, 19, 20, 21, 22, 23, 24]

X_train = train[~((train['month'] == 8) & (train['day'].isin(days)))].drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)'])
y_train = train[~((train['month'] == 8) & (train['day'].isin(days)))]['전력소비량(kWh)']

X_test = train[(train['month'] == 8) & (train['day'].isin(days))].drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)'])
y_test = train[(train['month'] == 8) & (train['day'].isin(days))]['전력소비량(kWh)']


print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(185730, 30) (16800, 30) (185730,) (16800,)


In [16]:
#결측값을 0으로 채웁니다
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

## Regression Model Fit

In [17]:
#!pip install optuna
import optuna

In [23]:
def objective(trial):
    params = {
        "random_state": 42,
        "n_estimators": trial.suggest_int("n_estimators", 70, 200),
        "objective": "reg:absoluteerror",
        "max_depth": trial.suggest_int("max_depth", 5, 25),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2)
    }
    
    model = XGBRegressor(**params)
    model.set_params(**{'objective':smape_model()})
    print(model)
    model.fit(temp_X_train, temp_y_train)
    y_preds = model.predict(temp_X_test)
    return smape(temp_y_test, y_preds)

In [24]:
%%time 
metrics_type = pd.DataFrame()
for i in train['건물유형'].unique():
    temp_X_train = X_train[X_train['건물유형'] == i].drop(['건물유형'], axis=1)
    temp_y_train = y_train.loc[temp_X_train.index]
    temp_X_test = X_test[X_test['건물유형'] == i].drop(['건물유형'], axis=1)
    temp_y_test = y_test.loc[temp_X_test.index]
    
#     model = XGBRegressor(random_state=42)
#     model.set_params(**{'objective':smape_model()})
#     model.fit(temp_X_train, temp_y_train)
#     y_preds = model.predict(temp_X_test)
    study = optuna.create_study()
    study.optimize(objective, n_trials=10)
    optuna.logging.set_verbosity(optuna.logging.CRITICAL)
    temp = pd.DataFrame([i, study.best_value, study.best_trial.params]).T
    temp.columns = ['건물유형', '유형별_score', '유형별_param']    
#     temp = pd.DataFrame([i, smape(temp_y_test, y_preds)]).T
#     temp.columns = ['건물유형', '유형별_score']
    metrics_type = metrics_type.append(temp)
#     display(temp)
#     vis_feature_importances_(model, temp_X_train)    
    

[I 2023-08-27 12:24:17,288] A new study created in memory with name: no-name-446343c7-89e7-4fea-baff-3e3547fbf073


XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.013339582297452041, max_delta_step=None,
             max_depth=9, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=158, n_jobs=None,
             num_parallel_tree=None, objective=None, random_state=42,
             reg_alpha=None, reg_lambda=None, scale_pos_weight=None,
             subsample=None, tree_method=None, validate_parameters=None,
             verbosity=None)


[I 2023-08-27 12:24:21,412] Trial 0 finished with value: 13.4750530350635 and parameters: {'n_estimators': 158, 'max_depth': 9, 'learning_rate': 0.013339582297452041}. Best is trial 0 with value: 13.4750530350635.


XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.11703079684168685, max_delta_step=None,
             max_depth=24, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=148, n_jobs=None,
             num_parallel_tree=None, objective=None, random_state=42,
             reg_alpha=None, reg_lambda=None, scale_pos_weight=None,
             subsample=None, tree_method=None, validate_parameters=None,
             verbosity=None)


[I 2023-08-27 12:24:42,746] Trial 1 finished with value: 6.911843002086597 and parameters: {'n_estimators': 148, 'max_depth': 24, 'learning_rate': 0.11703079684168685}. Best is trial 1 with value: 6.911843002086597.


...

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.14432602822598167, max_delta_step=None,
             max_depth=7, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, objective=None, random_state=42,
             reg_alpha=None, reg_lambda=None, scale_pos_weight=None,
             subsample=None, tree_method=None, validate_parameters=None,
             verbosity=None)
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.020694919866641907, max_delta_step=None,
             max_depth=17, min_child_weight=None,

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.16217296391091857, max_delta_step=None,
             max_depth=18, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=70, n_jobs=None,
             num_parallel_tree=None, objective=None, random_state=42,
             reg_alpha=None, reg_lambda=None, scale_pos_weight=None,
             subsample=None, tree_method=None, validate_parameters=None,
             verbosity=None)
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.024604185425819304, max_delta_step=None,
             max_depth=9, min_child_weight=None, 

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.09322712245794257, max_delta_step=None,
             max_depth=13, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=113, n_jobs=None,
             num_parallel_tree=None, objective=None, random_state=42,
             reg_alpha=None, reg_lambda=None, scale_pos_weight=None,
             subsample=None, tree_method=None, validate_parameters=None,
             verbosity=None)
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.019495289863724417, max_delta_step=None,
             max_depth=25, min_child_weight=None

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.06231746511523285, max_delta_step=None,
             max_depth=19, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=72, n_jobs=None,
             num_parallel_tree=None, objective=None, random_state=42,
             reg_alpha=None, reg_lambda=None, scale_pos_weight=None,
             subsample=None, tree_method=None, validate_parameters=None,
             verbosity=None)
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.07027507672607029, max_delta_step=None,
             max_depth=14, min_child_weight=None, 

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.11427488300969191, max_delta_step=None,
             max_depth=6, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=131, n_jobs=None,
             num_parallel_tree=None, objective=None, random_state=42,
             reg_alpha=None, reg_lambda=None, scale_pos_weight=None,
             subsample=None, tree_method=None, validate_parameters=None,
             verbosity=None)
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.14085637983805485, max_delta_step=None,
             max_depth=16, min_child_weight=None, 

In [25]:
%%time 
metrics_bno = pd.DataFrame()
for i in train['건물번호'].unique():

    temp_X_train = X_train[X_train['건물번호'] == i].drop(['건물번호','건물유형'], axis=1)
    temp_y_train = y_train.loc[temp_X_train.index]
    temp_X_test = X_test[X_test['건물번호'] == i].drop(['건물번호','건물유형'], axis=1)
    temp_y_test = y_test.loc[temp_X_test.index]
        
#     model = XGBRegressor(random_state=42)
#     model.set_params(**{'objective':smape_model()})
#     model.fit(temp_X_train, temp_y_train)
#     y_preds = model.predict(temp_X_test)

    study = optuna.create_study()
    study.optimize(objective, n_trials=10)
    optuna.logging.set_verbosity(optuna.logging.CRITICAL)
    temp = pd.DataFrame([i, study.best_value, study.best_trial.params]).T
    temp.columns = ['건물번호', '건물별_score', '건물별_param']
    
    metrics_bno = metrics_bno.append(temp)
#     display(temp)
#     vis_feature_importances_(model, temp_X_train)    
#     print()

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.08381074403439769, max_delta_step=None,
             max_depth=24, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=104, n_jobs=None,
             num_parallel_tree=None, objective=None, random_state=42,
             reg_alpha=None, reg_lambda=None, scale_pos_weight=None,
             subsample=None, tree_method=None, validate_parameters=None,
             verbosity=None)
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.10006205819682804, max_delta_step=None,
             max_depth=20, min_child_weight=None,

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.11864807063984728, max_delta_step=None,
             max_depth=22, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=134, n_jobs=None,
             num_parallel_tree=None, objective=None, random_state=42,
             reg_alpha=None, reg_lambda=None, scale_pos_weight=None,
             subsample=None, tree_method=None, validate_parameters=None,
             verbosity=None)
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.09821286495816753, max_delta_step=None,
             max_depth=21, min_child_weight=None,

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.1720638648154755, max_delta_step=None,
             max_depth=19, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=104, n_jobs=None,
             num_parallel_tree=None, objective=None, random_state=42,
             reg_alpha=None, reg_lambda=None, scale_pos_weight=None,
             subsample=None, tree_method=None, validate_parameters=None,
             verbosity=None)
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.09375382422471192, max_delta_step=None,
             max_depth=12, min_child_weight=None, 

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.18730248371645525, max_delta_step=None,
             max_depth=25, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=163, n_jobs=None,
             num_parallel_tree=None, objective=None, random_state=42,
             reg_alpha=None, reg_lambda=None, scale_pos_weight=None,
             subsample=None, tree_method=None, validate_parameters=None,
             verbosity=None)
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.043675145037390554, max_delta_step=None,
             max_depth=9, min_child_weight=None,

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.14387710898841413, max_delta_step=None,
             max_depth=15, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=80, n_jobs=None,
             num_parallel_tree=None, objective=None, random_state=42,
             reg_alpha=None, reg_lambda=None, scale_pos_weight=None,
             subsample=None, tree_method=None, validate_parameters=None,
             verbosity=None)
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.044496016340493466, max_delta_step=None,
             max_depth=22, min_child_weight=None,

...

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.152438591579685, max_delta_step=None, max_depth=25,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=82, n_jobs=None, num_parallel_tree=None,
             objective=None, random_state=42, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=None)
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.18561708343927436, max_delta_step=None,
             max_depth=14, min_child_weight=None, missing=nan,
  

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.15565840097720068, max_delta_step=None,
             max_depth=10, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=141, n_jobs=None,
             num_parallel_tree=None, objective=None, random_state=42,
             reg_alpha=None, reg_lambda=None, scale_pos_weight=None,
             subsample=None, tree_method=None, validate_parameters=None,
             verbosity=None)
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.020453510208844852, max_delta_step=None,
             max_depth=14, min_child_weight=None

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.18585152631247343, max_delta_step=None,
             max_depth=23, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=86, n_jobs=None,
             num_parallel_tree=None, objective=None, random_state=42,
             reg_alpha=None, reg_lambda=None, scale_pos_weight=None,
             subsample=None, tree_method=None, validate_parameters=None,
             verbosity=None)
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.1280755034714397, max_delta_step=None, max_depth=8,
             min_child_weight=None, mi

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.056942114465303936, max_delta_step=None,
             max_depth=25, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=83, n_jobs=None,
             num_parallel_tree=None, objective=None, random_state=42,
             reg_alpha=None, reg_lambda=None, scale_pos_weight=None,
             subsample=None, tree_method=None, validate_parameters=None,
             verbosity=None)
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.07174535278111026, max_delta_step=None,
             max_depth=22, min_child_weight=None,

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.03765685484957083, max_delta_step=None,
             max_depth=23, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=177, n_jobs=None,
             num_parallel_tree=None, objective=None, random_state=42,
             reg_alpha=None, reg_lambda=None, scale_pos_weight=None,
             subsample=None, tree_method=None, validate_parameters=None,
             verbosity=None)
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.12025147347561937, max_delta_step=None,
             max_depth=20, min_child_weight=None,

In [26]:
metrics_bno['건물번호'] = metrics_bno['건물번호'].astype(int)

In [27]:
metrics_bno = metrics_bno.merge(building_info[['건물유형','건물번호']], on=['건물번호'])

In [28]:
metrics = metrics_bno.merge(metrics_type, on=['건물유형'])
metrics[:4]

,건물번호,건물별_score,건물별_param,건물유형,유형별_score,유형별_param
0,1,6.074664,"{'n_estimators': 109, 'max_depth': 17, 'learni...",건물기타,6.78398,"{'n_estimators': 199, 'max_depth': 20, 'learni..."
1,2,7.885423,"{'n_estimators': 128, 'max_depth': 5, 'learnin...",건물기타,6.78398,"{'n_estimators': 199, 'max_depth': 20, 'learni..."
2,3,9.197367,"{'n_estimators': 150, 'max_depth': 7, 'learnin...",건물기타,6.78398,"{'n_estimators': 199, 'max_depth': 20, 'learni..."
3,4,4.029241,"{'n_estimators': 94, 'max_depth': 14, 'learnin...",건물기타,6.78398,"{'n_estimators': 199, 'max_depth': 20, 'learni..."


In [29]:
# 유형보다 건물이 더 좋은 스코어를 가지는 건물번호 list
bno = metrics[metrics['건물별_score'] < metrics['유형별_score']]['건물번호'].values
len(bno), bno

(62,
 array([  1,   4,   5,   6,   8,   9,  11,  12,  15,  16,  22,  23,  24,
         25,  26,  27,  32,  33,  37,  38,  39,  41,  42,  44,  45,  48,
         49,  50,  55,  56,  57,  58,  59,  60,  61,  62,  64,  68,  69,
         70,  71,  73,  75,  76,  77,  78,  79,  81,  83,  84,  85,  86,
         88,  89,  90,  92,  93,  94,  96,  97,  99, 100]))

In [30]:
metrics_type['유형별_score'].mean(), metrics_bno['건물별_score'].mean()
# (7.026821963284757, 6.107093822048833)

(5.003717183111456, 5.0182601290713755)

In [31]:
# 가중 평균 
metrics_type['유형별_score'].mean() * ((100-len(bno))/100) + metrics_bno['건물별_score'].mean() * (len(bno)/100)

5.0127338096066065

In [ ]:
# 5.24858577401958
# 5.012733809606606

In [ ]:
metrics['score'] = metrics['유형별_score'] 
metrics.loc[metrics['건물번호'].isin(bno), 'score'] = metrics['건물별_score']

In [ ]:
metrics['건물이낫다'] = metrics['건물별_score'] < metrics['유형별_score']

In [ ]:
# metrics.to_excel('오차큰것부터_20230810-3.xlsx')

## Inference & Submit

In [32]:
%%time
result_type = pd.DataFrame()
for i in train['건물유형'].unique():
    
    temp_X_train = train[train['건물유형'] == i].drop(['건물유형','num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)'], axis=1)
    temp_y_train = train.loc[temp_X_train.index][['전력소비량(kWh)']]
    
    model = XGBRegressor(n_estimators=metrics_type[metrics_type['건물유형'] == i]['유형별_param'][0]['n_estimators']
                         ,max_depth=metrics_type[metrics_type['건물유형'] == i]['유형별_param'][0]['max_depth']
                         ,learning_rate=metrics_type[metrics_type['건물유형'] == i]['유형별_param'][0]['learning_rate']
                         ,random_state=42)
    print(model)
    model.set_params(**{'objective':smape_model()})
    
    model.fit(temp_X_train, temp_y_train)
                                                 
    temp_test = test[test['건물유형'] == i].drop(['건물유형','num_date_time', '일시'], axis=1)
    y_preds = model.predict(temp_test)
    result_type = result_type.append( pd.concat([test[test['건물유형'] == i].reset_index(), pd.DataFrame(y_preds, columns=['answer_유형별']).reset_index()], axis=1) )
    
result_type.shape    

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.03184660488502149, max_delta_step=None,
             max_depth=20, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=199, n_jobs=None,
             num_parallel_tree=None, random_state=42, reg_alpha=None,
             reg_lambda=None, scale_pos_weight=None, subsample=None,
             tree_method=None, validate_parameters=None, verbosity=None)
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.16641811964323008, max_delta_step=None,
             max_depth=23, min_child_weight=None, missing=nan,
             mo

(16800, 35)

In [33]:
%%time
result_bno = pd.DataFrame()
for i in train['건물번호'].unique():
    
    temp_X_train = train[train['건물번호'] == i].drop(['건물번호','건물유형','num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)'], axis=1)
    temp_y_train = train.loc[temp_X_train.index][['전력소비량(kWh)']]
    
    model = XGBRegressor(n_estimators=metrics_bno[metrics_bno['건물번호'] == i]['건물별_param'].values[0]['n_estimators']
                         , max_depth=metrics_bno[metrics_bno['건물번호'] == i]['건물별_param'].values[0]['max_depth']
                         , learning_rate=metrics_bno[metrics_bno['건물번호'] == i]['건물별_param'].values[0]['learning_rate']
                         , random_state=42)
    print(model)
    model.set_params(**{'objective':smape_model()})
    
    model.fit(temp_X_train, temp_y_train)
                                                 
    temp_test = test[test['건물번호'] == i].drop(['건물번호','건물유형','num_date_time', '일시'], axis=1)
    y_preds = model.predict(temp_test)
    result_bno = result_bno.append( pd.concat([test[test['건물번호'] == i].reset_index(), pd.DataFrame(y_preds, columns=['answer_건물별']).reset_index()], axis=1) )
    
result_bno.shape    

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.16067696380712257, max_delta_step=None,
             max_depth=17, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=109, n_jobs=None,
             num_parallel_tree=None, random_state=42, reg_alpha=None,
             reg_lambda=None, scale_pos_weight=None, subsample=None,
             tree_method=None, validate_parameters=None, verbosity=None)
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.06662219040123424, max_delta_step=None,
             max_depth=5, min_child_weight=None, missing=nan,
             mon

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.030333163928900786, max_delta_step=None,
             max_depth=20, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=83, n_jobs=None,
             num_parallel_tree=None, random_state=42, reg_alpha=None,
             reg_lambda=None, scale_pos_weight=None, subsample=None,
             tree_method=None, validate_parameters=None, verbosity=None)
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.1284618856273007, max_delta_step=None, max_depth=9,
             min_child_weight=None, missing=nan, monotone_constrai

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.16472269159757294, max_delta_step=None,
             max_depth=7, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=103, n_jobs=None,
             num_parallel_tree=None, random_state=42, reg_alpha=None,
             reg_lambda=None, scale_pos_weight=None, subsample=None,
             tree_method=None, validate_parameters=None, verbosity=None)
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.04256861228000304, max_delta_step=None,
             max_depth=9, min_child_weight=None, missing=nan,
             mono

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.11335225988503042, max_delta_step=None,
             max_depth=6, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=95, n_jobs=None,
             num_parallel_tree=None, random_state=42, reg_alpha=None,
             reg_lambda=None, scale_pos_weight=None, subsample=None,
             tree_method=None, validate_parameters=None, verbosity=None)
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.1041853742952292, max_delta_step=None, max_depth=8,
             min_child_weight=None, missing=nan, monotone_constraint

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.11229955884662901, max_delta_step=None,
             max_depth=20, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=140, n_jobs=None,
             num_parallel_tree=None, random_state=42, reg_alpha=None,
             reg_lambda=None, scale_pos_weight=None, subsample=None,
             tree_method=None, validate_parameters=None, verbosity=None)
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.10453814783870995, max_delta_step=None,
             max_depth=9, min_child_weight=None, missing=nan,
             mon

             tree_method=None, validate_parameters=None, verbosity=None)
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.1525457305831791, max_delta_step=None, max_depth=9,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=159, n_jobs=None, num_parallel_tree=None,
             random_state=42, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=None)
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.1912838449238827, max_delta_step=None,
        

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.14751032614011106, max_delta_step=None,
             max_depth=9, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=170, n_jobs=None,
             num_parallel_tree=None, random_state=42, reg_alpha=None,
             reg_lambda=None, scale_pos_weight=None, subsample=None,
             tree_method=None, validate_parameters=None, verbosity=None)
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.1157869306687436, max_delta_step=None, max_depth=9,
             min_child_weight=None, missing=nan, monotone_constrain

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.04792105337597298, max_delta_step=None,
             max_depth=5, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=169, n_jobs=None,
             num_parallel_tree=None, random_state=42, reg_alpha=None,
             reg_lambda=None, scale_pos_weight=None, subsample=None,
             tree_method=None, validate_parameters=None, verbosity=None)
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.1352641135388917, max_delta_step=None, max_depth=6,
             min_child_weight=None, missing=nan, monotone_constrain

(16800, 35)

In [34]:
result = result_type[['num_date_time','건물번호','answer_유형별','weekday', 'time']].merge(result_bno[['num_date_time','건물번호','answer_건물별']], on=['num_date_time','건물번호'])
result['answer'] = result['answer_유형별'] 
result.loc[result['건물번호'].isin(bno), 'answer'] = result['answer_건물별']

# 후처리 
postprocessing = train.groupby(['건물번호', 'weekday','time'])['전력소비량(kWh)'].min().reset_index()
result = result.merge(postprocessing, on=['건물번호', 'weekday','time'])
result.loc[result['answer'] < result['전력소비량(kWh)'], 'answer'] = result['전력소비량(kWh)']

result[['num_date_time','answer']].to_csv('20230827-2_x.csv', index=False)